In [10]:
from tqdm.auto import tqdm
import requests 

/home/codespace/.python/current/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [12]:
query = "How do I execute a command in a running docker container?"

In [13]:
from elasticsearch import Elasticsearch
es_client = Elasticsearch('http://localhost:9200')

In [14]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = "course_questions"

es_client.indices.create(index=index_name, body=index_settings)
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|██████████| 948/948 [00:25<00:00, 37.78it/s]


In [22]:
search_query = {
"size": 5,
"query": {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["question^4", "text"],
                "type": "best_fields"
            }
        },
    }
}
}
index_name = "course_questions"
response = es_client.search(index=index_name, body=search_query)

In [32]:
response['hits']['hits'][0]['_score']

84.050095

In [33]:
search_query = {
"size": 3,
"query": {
    "bool": {
        "must": {
            "multi_match": {
                "query": query,
                "fields": ["question^4", "text"],
                "type": "best_fields"
            }
        },
        "filter": {
            "term": {
                "course": "machine-learning-zoomcamp"
            }
        }
    }
}
}
response = es_client.search(index=index_name, body=search_query)

In [35]:
response['hits']['hits']

[{'_index': 'course_questions',
  '_id': 'icL1TpABjxdZNtJyzcf_',
  '_score': 84.050095,
  '_source': {'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
   'section': '5. Deploying Machine Learning Models',
   'question': 'How do I debug a docker container?',
   'course': 'machine-learning-zoomcamp'}},
 {'_index': 'course_questions',
  '_id': 'qML1TpABjxdZNtJy0cdE',
  '_score': 51.04628,
  '_source': {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_

In [42]:
context_template = """
Q: {question}
A: {text}
""".strip()
context = ""
for resp in response['hits']['hits']:
    context += context_template.format(question=resp['_source']['question'], text=resp['_source']['text'])
    context += '\n\n'
context = context.strip()

In [43]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT:
{context}
""".strip()
prompt = prompt_template.format(question=query, context=context)

In [44]:
len(prompt)

1462

In [46]:
import tiktoken
encoding = tiktoken.encoding_for_model("gpt-4o")

In [49]:
len(encoding.encode(prompt))

322